https://www.tensorflow.org/get_started/get_started

# Getting Started With TensorFlow 
https://www.tensorflow.org/get_started/get_started

The lowest level API--TensorFlow Core-- provides you with complete programming control.  
The higher level APIs are built on top of TensorFlow Core, like the high-level API tf.contrib.learn 

## Tensors

A tensor consists of a set of primitive values shaped into an array of numbers.  
A tensor's rank is its number of dimensions.

### TensorFlow Core tutorial

#### Importing TensorFlow

In [22]:
import tensorflow as tf
import numpy as np

#### The Computational Graph

Tensorflow core programs consists of  
&nbsp;&nbsp;&nbsp;&nbsp;Building computational graph  
&nbsp;&nbsp;&nbsp;&nbsp;Running computational graph  

Computational graph is a series of TensorFlow operations arranged into a graph of nodes.  
&nbsp;&nbsp;&nbsp;&nbsp;Each node takes zero or more tensors as inputs and produces a tensor as an output  

The constant node, contains no input, and out the value stored internally

In [19]:
node1 = tf.constant(3.0, tf.float32)
node2 = tf.constant(4.0)  #shape = (); same as numpy

They are just nodes that will only produce values when evaluated (not when barely printed)

In [20]:
print (node1,node2)  #Notice that printing the nodes does not output the values 3.0 and 4.0 as you might expect

Tensor("Const_10:0", shape=(), dtype=float32) Tensor("Const_11:0", shape=(), dtype=float32)


For the nodes to be evaluated, a session needs to be run.  
&nbsp;&nbsp;&nbsp;&nbsp;a session encapsulates the control and state of the TensorFlow runtime.

In [42]:
sess = tf.Session()
sess.run([[node1],[node2]])

[[3.0], [4.0]]

Create more complicated computations by combining Tensor nodes with operation (Operations are also nodes.)

In [44]:
node3 = tf.add(node1, node2)
print("node3: ", node3)
print("sess.run(node3): ",sess.run(node3))

node3:  Tensor("Add_6:0", shape=(), dtype=float32)
sess.run(node3):  7.0


In [ ]:
#^?^ how to produce the TensorBoard graph? [20170417]

In [ ]:
Placeholder is a promise to provide a value later, which is used to accept external values, just like the input parameter

In [35]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = tf.add(a,b)

Use the *feed_dict* parameter to specify Tensors that provide concrete values to these placeholders

In [37]:
sess.run(adder_node,{a: 3, b: 4.5})

7.5

In [38]:
sess.run(adder_node,{a: [1,3], b: [2,4]})

array([ 3.,  7.], dtype=float32)

A more complicated one

In [51]:
add_and_triple = adder_node * 3
sess.run(add_and_triple, {a: 3,b: 4.5})

22.5

To make the model trainable, we need to be able to modify the graph to get new outputs with the same input, thus the introduction of Variables

In [66]:
# W = tf.Variable([-1.], tf.float32) #exactly the W and b that could eliminate the 
# b = tf.Variable([1.], tf.float32)
W = tf.Variable([.3], tf.float32) 
b = tf.Variable([-.3], tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

Constants are initialized when you call tf.constant, and their value can never change.   
Variables are initialized when explicitly calling a special operation.

In [94]:
init = tf.global_variables_initializer() 
sess.run(init)

init is a handle to the TensorFlow sub-graph that initializes all the global variables

Evaluate linear_model for several values of x(a placeholder) simultaneously

In [96]:
print(sess.run(linear_model, {x:[1.,2,3,4]}))


TypeError: unhashable type: 'numpy.ndarray'

To evaluate the model on training data, a loss function, which measures how far apart the current model is from the provided data, is needed.  

Standard loss model for linear regression

In [69]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]})

0.0

*tf.Variable* but can be changed using operations like tf.assign

In [71]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb]) #equal to sess.run(init) in a sense
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))


0.0


### tf.train API

Optimizers that slowly change each variable in order to minimize the loss function.  
The simplest optimizer is gradient descent. It modifies each variable according to the magnitude of the derivative of loss with respect to that variable

In [77]:
#^?^ the function of tf.gradients?
optimizer = tf.train.GradientDescentOptimizer(0.01) #^?^ what does 0.01 stand for?
train = optimizer.minimize(loss)

In [78]:
sess.run(init)

In [82]:
for i in range(1000):
  sess.run(train, {x:[1,2,3,4], y:[0,-1,-2,-3]})

In [85]:
sess.run((W,b))

(array([-1.], dtype=float32), array([ 1.], dtype=float32))

In [ ]:
# ^?^ the TensorBoard graph generated?

### tf.contrib.learn

*tf.contrib.learn* is a high-level TensorFlow library that simplifies the mechanics of machine learning,  
&nbsp;&nbsp;&nbsp;&nbsp;running training loops  
&nbsp;&nbsp;&nbsp;&nbsp;running evaluation loops  
&nbsp;&nbsp;&nbsp;&nbsp;managing data sets  
&nbsp;&nbsp;&nbsp;&nbsp;managing feeding  


#### Basic usage

In [88]:
import tensorflow as tf
# NumPy is often used to load, manipulate and preprocess data.
import numpy as np

In [89]:
# Declare list of features. We only have one real-valued feature. There are many
# other types of columns that are more complicated and useful.
features = [tf.contrib.layers.real_valued_column("x", dimension=1)]

In [ ]:
# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# logistic regression, linear classification, logistic classification, and
# many neural network classifiers and regressors. The following code
# provides an estimator that does linear regression.
estimator = tf.contrib.learn.LinearRegressor(feature_columns=features)

In [90]:
# TensorFlow provides many helper methods to read and set up data sets.
# Here we use `numpy_input_fn`. We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
x = np.array([1., 2., 3., 4.])
y = np.array([0., -1., -2., -3.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x":x}, y, batch_size=4,
                                              num_epochs=1000)

In [91]:
# We can invoke 1000 training steps by invoking the `fit` method and passing the
# training data set.
estimator.fit(input_fn=input_fn, steps=1000) #^?^ what kind of method is used to minimize the losses?

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.


Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.


INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Saving checkpoints for 1001 into /var/folders/lt/zpqkqs_n63j0g2bpkbnxtkmh0000gn/T/tmp8f8side7/model.ckpt.


...

INFO:tensorflow:Saving checkpoints for 2000 into /var/folders/lt/zpqkqs_n63j0g2bpkbnxtkmh0000gn/T/tmp8f8side7/model.ckpt.


INFO:tensorflow:Saving checkpoints for 2000 into /var/folders/lt/zpqkqs_n63j0g2bpkbnxtkmh0000gn/T/tmp8f8side7/model.ckpt.


INFO:tensorflow:Loss for final step: 1.13687e-13.


INFO:tensorflow:Loss for final step: 1.13687e-13.


LinearRegressor(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead object at 0x112f2ca90>, 'feature_columns': [_RealValuedColumn(column_name='x', dimension=1, default_value=None, dtype=tf.float32, normalizer=None)], 'joint_weights': False, 'optimizer': None, 'gradient_clip_norm': None})

In [92]:
# Here we evaluate how well our model did. In a real example, we would want
# to use a separate validation and testing data set to avoid overfitting.
estimator.evaluate(input_fn=input_fn)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.


Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.


INFO:tensorflow:Starting evaluation at 2017-04-17-12:44:14


INFO:tensorflow:Starting evaluation at 2017-04-17-12:44:14


INFO:tensorflow:Finished evaluation at 2017-04-17-12:44:15


INFO:tensorflow:Finished evaluation at 2017-04-17-12:44:15


INFO:tensorflow:Saving dict for global step 2000: global_step = 2000, loss = 3.4072e-13


INFO:tensorflow:Saving dict for global step 2000: global_step = 2000, loss = 3.4072e-13


{'global_step': 2000, 'loss': 3.407198e-13}

### A custom model

*tf.contrib.learn.LinearRegressor* is actually a sub-class of *tf.contrib.learn.Estimator*.  
Instead of sub-classing *Estimator*, we simply provide *Estimator* a function *model_fn* that tells *tf.contrib.learn* how it can evaluate predictions, training steps, and loss. The code is as follows:

In [ ]:
import numpy as np
import tensorflow as tf

In [93]:
# Declare list of features, we only have one real-valued feature
def model(features, labels, mode):
  # Build a linear model and predict values
  W = tf.get_variable("W", [1], dtype=tf.float64)
  b = tf.get_variable("b", [1], dtype=tf.float64)
  y = W*features['x'] + b
  # Loss sub-graph
  loss = tf.reduce_sum(tf.square(y - labels))
  # Training sub-graph
  global_step = tf.train.get_global_step()
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
  # ModelFnOps connects subgraphs we built to the
  # appropriate functionality.
  return tf.contrib.learn.ModelFnOps(
      mode=mode, predictions=y,
      loss=loss,
      train_op=train)

In [ ]:
estimator = tf.contrib.learn.Estimator(model_fn=model)

In [102]:
# define our data set
x = np.array([1., 2., 3., 4.])
y = np.array([0., -1., -2., -3.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x": x}, y, 4, num_epochs=1000)


In [103]:
# train
estimator.fit(input_fn=input_fn, steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Saving checkpoints for 2001 into /var/folders/lt/zpqkqs_n63j0g2bpkbnxtkmh0000gn/T/tmpp9mhfqlg/model.ckpt.


INFO:tensorflow:Saving checkpoints for 2001 into /var/folders/lt/zpqkqs_n63j0g2bpkbnxtkmh0000gn/T/tmpp9mhfqlg/model.ckpt.


INFO:tensorflow:loss = 3.16965909161e-24, step = 2001


...

INFO:tensorflow:Saving checkpoints for 3000 into /var/folders/lt/zpqkqs_n63j0g2bpkbnxtkmh0000gn/T/tmpp9mhfqlg/model.ckpt.


INFO:tensorflow:Saving checkpoints for 3000 into /var/folders/lt/zpqkqs_n63j0g2bpkbnxtkmh0000gn/T/tmpp9mhfqlg/model.ckpt.


INFO:tensorflow:Loss for final step: 3.59917788007e-30.


INFO:tensorflow:Loss for final step: 3.59917788007e-30.


Estimator(params=None)

In [101]:
# evaluate our model
print(estimator.evaluate(input_fn=input_fn, steps=10))

INFO:tensorflow:Starting evaluation at 2017-04-17-13:28:13


INFO:tensorflow:Starting evaluation at 2017-04-17-13:28:13


INFO:tensorflow:Evaluation [1/10]


INFO:tensorflow:Evaluation [1/10]


INFO:tensorflow:Evaluation [2/10]


...

INFO:tensorflow:Finished evaluation at 2017-04-17-13:28:13


INFO:tensorflow:Finished evaluation at 2017-04-17-13:28:13


INFO:tensorflow:Saving dict for global step 2000: global_step = 2000, loss = 3.62561e-23


INFO:tensorflow:Saving dict for global step 2000: global_step = 2000, loss = 3.62561e-23


{'loss': 3.6256107e-23, 'global_step': 2000}
